Importer les packages

In [79]:
# importer les modules python nécessaires
from itertools import product

import re
import ViennaRNA

# Pour lire le fichier fasta
from Bio import SeqIO

import subprocess

# importer numpy
import numpy as np

# importer les modules scikit-learn nécessaires


1. Repliement avec RNAfold

In [71]:
# À COMPLÉTER cette fonction

def executer_rnafold(fichier_fasta, fichier_output):
    with open(fichier_output, "w") as output_file:
        for seq_record in SeqIO.parse(fichier_fasta, "fasta"):
            sequence = str(seq_record.seq)
            process = subprocess.Popen(['RNAfold'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            [stdout, stderr] = process.communicate(input=sequence)
            output_file.write(f">{seq_record.id}\n")  # Write the sequence name
            output_file.write(f"{stdout.split()[0]}\n{stdout.split()[1]}\n")  # Write the RNAfold output
        
        print(f"{stdout.split()[0]}\n{stdout.split()[1]}\n")

In [72]:
executer_rnafold("Fasta/hsa_hairpin.fasta", "Fasta/fichier_output.txt")

UCUCCGUUUAUCCCACCACUGCCACCAUUAUUGCUACUGUUCAGCAGGUGCUGCUGGUGGUGAUGGUGAUAGUCUGGUGGGGGCGGUGG
.(.((((...(((((((((((.((((((((((((.......((((((...)))))))))))))))))).))))..))))))))))).).



In [73]:
from collections import defaultdict

def traiter_fasta_fold(fichier_fold):
    """
    La fonction traite les données du fichier fasta des repliements de RNAfold pour les organiser
    en un dictionnaire de listes contenant les séquences et les structures secondaires

    paramètres :
    ----------
        fichier_fold : (string) chemin/nom du fichier fasta contenant les repliements de RNAfold
    
    Retourne :
    --------
        repliements : (defaultdict(list)) contenant les séquences et leurs repliements
                de forme : dict[nom] = [séquence, structure]
    """
    repliements = defaultdict(list)

    # 1. Lire le fichier fichier_fold et récupérer son contenu dans une liste lignes
    with open(fichier_fold, 'r') as file:
        lignes = file.readlines()

    # Il faut supprimer "\n" à la fin des lignes
    lignes_propres = [ligne.strip() for ligne in lignes]

    # 2. Traiter repliement_fasta pour créer un dictionnaire de listes defaultdict(list)
    # avec la forme :
    # dict[nom] = [séquence, structure] où nom est l'identifiant de la séquence
    # Remarque : les séquences doivent être converties en minuscule (lowercase)
    i = 0
    while i < len(lignes_propres):
        if lignes_propres[i].startswith('>'):
            nom_sequence = lignes_propres[i][1:]
            sequence = lignes_propres[i+1].lower()
            structure = lignes_propres[i+2]
            repliements[nom_sequence] = [sequence, structure]
            i += 3
        else:
            i += 1

    return repliements


In [83]:
repliments = traiter_fasta_fold("Fasta/fichier_output.txt")
print(repliments.items())

dict_items([('hsa-mir-5683', ['ggagcuuguuacagaugcagauucucugacuucuuacugcaccagugaagucaggaucugcauuugaauaagaccc', '((..((((((.(((((((((((.(.((((((((..((((...))))))))))))))))))))))))))))))..))']), ('hsa-mir-1587', ['uuugggcugggcuggguugggcaguucuucugcuggacucaccugugaccagc', '.((((..((((.((((((.(((((.....))))).))))))))))...)))).']), ('hsa-mir-572', ['gucgaggccguggcccggaaguggucggggccgcugcgggcggaagggcgccugugcuucguccgcucggcgguggcccagccaggcccgcggga', '.......((((((.((.....((((..(((((((((((((((((.((((......))))..))))))).)))))))))).)))))).))))))..']), ('hsa-mir-2116', ['gaccuaggcuagggguucuuagcauaggaggucuucccaugcuaagaaguccucccaugccaagaacucccagacuagga', '..(((((.((.(((((((((.((((.(((((.((((.........)))).))))).)))).))))))))).)).))))).']), ('hsa-mir-3675', ['ggaugauaaguuauggggcuucuguagagauuucuaugagaacaucucuaaggaacucccccaaacugaauuc', '((((....((((.(((((.((((.(((((((((((...)))).))))))).))))...)))))))))..))))']), ('hsa-mir-6724-3', ['cgcugcgcuucugggcccgcggcgggcguggggcugcccgggccggucgaccagcgcgccguagcucccgaggc

2. Calcul du nombre d’occurrences des triplets

In [75]:
# Ne pas modifier la fonction suivante :

def get_all_triplets():
    """
    La fonction génère l'ensemble des triplets
    """

    db = ["...", "..(", ".((", "(((", "((.", "(..", "(.(", ".(."]
    nd = ["a", "c", "g", "u"]

    return [i+j for i,j in product(nd, db)]

In [76]:
# Generer les seauence RNQ et leur structures
triplets = get_all_triplets()

In [80]:
# Ne pas modifier la fonction suivante :

def get_seq_triplets(sequence, structure):
    """
    La fonction calcule le nombre d'occurrences des triplets d'une séquence ARN

    paramètres :
    ----------
        sequence : (string) séquence nucleotidique
        structure : (string) structure secondaire dot-bracket

    Retourne :
    --------
        seq_triplets : (defaultdict(int)) dictionnaire contenant le nombre 
                d'occurrences des triplets dans (sequence, structure)
    """

    seq_triplets = defaultdict(int)

    assert len(sequence) == len(structure)

    struct = re.sub("\)", "(", structure)

    seq_tab = list(sequence)
    stc_tab = list(struct)

    for ind in range(len(seq_tab)):
        if ind == 0: 
            before = "."
        else: 
            before = stc_tab[ind-1]

        midlle = stc_tab[ind]

        if ind == len(seq_tab)-1: 
            after = "."
        else:
            after = stc_tab[ind+1]

        triplet = seq_tab[ind] + before + midlle + after
        seq_triplets[triplet] += 1

    return seq_triplets

In [98]:
# Calculer les occurences des triplets dans les sequences et structures
for sequence, structure in repliments.values():
    # print(len(sequence))
    # print(len(structure))
    # print(sequence)
    # print(structure)
    seq_triplets = get_seq_triplets(sequence, structure)
    # print(f"Occurrences of triplets in sequence {sequence}: {seq_triplets}")

In [85]:
# Ne pas modifier la fonction suivante :

def calculer_Xu_triplets(repliements):
    """
    La fonction calcule les valeurs des triplets à partir d'un ensemble de repliements 
    (séquences et structures secondaires)

    paramètres :
    ----------
        repliements : (defaultdict(list)) contenant les séquences et leurs repliements
                de forme dict[nom] = [séquence, structure]

    Retourne :
    --------
        triplets = (np.ndarray) de forme (nb_sequences, nb_triplets) contenant 
                les valeurs des triplets Xu pour chaque séquence dans repliements
                (nb_triplets = 32)
    """
    liste_triplets = get_all_triplets()

    nb_sequences = len(repliements)
    nb_triplets = len(liste_triplets)

    triplets = np.zeros((nb_sequences, nb_triplets))
    
    for i, nom in enumerate(repliements):
        seq_triplets = get_seq_triplets(repliements[nom][0], repliements[nom][1])
        
        for j, triplet in enumerate(liste_triplets):
            if triplet in seq_triplets:
                triplets[i, j] = seq_triplets[triplet]
    
    return triplets

In [86]:
# Cette cellule est dédiée pour tester calculer_Xu_triplets()
Xu_triplets = calculer_Xu_triplets(repliments)

array([[1., 0., 1., ..., 1., 2., 0.],
       [0., 1., 0., ..., 2., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       ...,
       [3., 2., 1., ..., 1., 1., 0.],
       [5., 1., 2., ..., 1., 1., 0.],
       [1., 1., 0., ..., 2., 2., 0.]])

3. Construction du jeu de données d'entrainement

In [87]:
# À COMPLÉTER cette fonction

def construire_dataset(data1, data2):
    """
    La fonction concatène et transforme les données d'entrée pour créer un jeu de données. 
    Elle construit aussi un tableau ndarray contenant les étiquettes (classes) de ces données.

    paramètres :
    ----------
        data1 : (np.ndarray) de forme (nb_sequences1, nb_triplets) contenant le nombre d'occurrence 
            de triplets de la première classe (par ex. classe positive)

        data2 : (np.ndarray) de forme (nb_sequences2, nb_triplets) contenant le nombre d'occurrence 
            de triplets de la deuxième classe (par ex. classe négative)

    Retourne :
    --------
        X : (np.ndarray) de forme (nb_sequences1+nb_sequences2, nb_triplets) contenant le nombre d'occurrence 
            de triplets dans tous le jeu de données (concaténation de data1 et data2)

        y : (np.ndarray) de forme (nb_sequences1+nb_sequences2) contenant les étiquettes (classes) des instances 
            selon ce schéma :
                Étiquettes des instances de data1 = 0
                Étiquettes des instances de data2 = 1
    """
    
    # 1. Construire X

    X = np.concatenate((data1, data2), axis=0)
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)

    # 2. Mettre à l'échelle les valeurs de X entre 0 et 1 avec la méthode min-max

    X = (X - X_min) / (X_max - X_min)

    # 3. Construire y
    
    y = np.concatenate((np.zeros(data1.shape[0]), np.ones(data2.shape[0])))

    assert X.shape[0] == y.shape[0]

    return X, y


In [99]:
# Est-ce on a besoin des deux autres fichiers d'hairpin ? 
# mais ils sont les deux de classe negative
# Data 1
executer_rnafold("Fasta/hsa_hairpin_neg1.fasta", "Fasta/repliments/hsa_hairpin_neg1_repliments.txt")
data1dict = traiter_fasta_fold("Fasta/repliments/hsa_hairpin_neg1_repliments.txt")
for sequence, structure in data1dict.values():
    seq_triplets = get_seq_triplets(sequence, structure)
    # print(f"Occurrences of triplets in sequence {sequence}: {seq_triplets}")
data1 = calculer_Xu_triplets(data1dict)


UGCUGAAAAUAGCCGACCUCCCGGGUAAUCCGGACACGCUCCGAUAGCCACCACCCAGAUGUUACCG
.((((....((.(((......))).))...((((.....)))).))))...................



In [90]:
executer_rnafold("Fasta/hsa_hairpin_neg2.fasta", "Fasta/repliments/hsa_hairpin_neg2_repliments.txt")
data2dict = traiter_fasta_fold("Fasta/repliments/hsa_hairpin_neg2_repliments.txt")
for sequence, structure in data1dict.values():
    seq_triplets = get_seq_triplets(sequence, structure)
    # print(f"Occurrences of triplets in sequence {sequence}: {seq_triplets}")
data2 = calculer_Xu_triplets(data1dict)

AUAUUGAACUCGAGUUGGAAGAGGCGAGUCCGGUCUCAAAGUGACGACUGGCCCCGCCUCUUCCUCUCGGUCCC
.......((.((((..((((((((((.(.(((((((((...))).)))))).).)))))))))).))))))...



In [91]:
x,y = construire_dataset(data1, data2)

4. Évaluation et entrainement des modèles supervisés

In [96]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score

def evaluer_cv_entrainer(x, y, cv=5, modele="lsvc", mesure='f1_weighted'):
    """
    La fonction évalue un classifieur de type (modele) avec validation croisée de (cv) itérations sur 
    le jeu de données (X, y). Elle utilise (mesure) comme mesure d'évaluation de performance.
    La fonction entraine et retourne le classifieur sur (X, y).
    
    paramètres :
    ----------
        X : (np.ndarray) de forme (nb_instances, nb_triplets) contenant le nombre d'occurrence 
            de triplets du jeu d'entrainement

        y : (np.ndarray) de forme (nb_instances) contenant les étiquettes (classes) des instances 
            selon ce schéma :
                Étiquettes des instances de classe 0 (positive) = 0
                Étiquettes des instances de classe 1 (négative) = 1
        cv : (int) nombre d'itération de la validation croisée
        modele : (string) type du modèle à entrainer. valeurs accéptées:
            lsvc = SVC linéaire
            nsvc = SVC non linéaire
            mnb = Naive Bayes multinomial
            gnb = Naive Bayes gaussien
            dtc = Arbre de décision        
        mesure : (string) type de mesure pour évaluer la performance du modèle. Valeurs acceptées :
            https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values
    Retourne :
    --------
        scores : (np.ndarray) liste contenant scores de (cv) itérations de validation croisée
        classifieur : (estimateur scikit-learn) modèle entrainé avec X et y
    """
    # Modèles acceptés :
    modeles = {
        "lsvc": SVC(kernel='linear'),
        "nsvc": SVC(kernel='rbf'),
        "mnb": MultinomialNB(),
        "gnb": GaussianNB(),
        "dtc": DecisionTreeClassifier()
    }
    
    if modele not in modeles:
        print("Type de modele {} est invalide".format(modele))
        modele = "lsvc"
        print("Le modele {} sera entrainé".format(modele))
    
    # 1. Initialiser une instance de classifieur selon la valeur de modele
    classifieur = modeles[modele]
    
    # 2. Calculer les scores de validation croisée du classifieur avec X et y
    scorer = make_scorer(f1_score, average=mesure)
    scores = cross_val_score(classifieur, x, y, cv=cv, scoring=scorer)
    
    # 3. Entrainer le classifieur avec X et y
    classifieur.fit(x, y)
    
    return scores, classifieur

In [97]:
# Évaluer et entraîner un classifieur avec validation croisée
scores, classifieur = evaluer_cv_entrainer(x, y, cv=5, modele="lsvc", mesure='f1_weighted')

c:\Users\P14s\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\P14s\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\P14s\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\P14s\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\_param_validation.py", line 203, in wrapper
    validate_parameter_constraints(
  File "c:\Users\P14s\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\uti

In [ ]:
# Step 1: Calculer les triplets à partir des repliements
repliements = ...  # Obtenez les repliements des séquences génomiques
triplets_repliements = calculer_Xu_triplets(repliements)

# Step 2: Obtenir les triplets à partir des séquences et des structures
sequence = ...  # Obtenez les séquences génomiques
structure = ...  # Obtenez les structures génomiques
triplets_seq_struct = get_seq_triplets(sequence, structure)

# Step 3: Traiter un fichier de repliement au format FASTA
fichier_fold = ...  # Chemin vers le fichier de repliement au format FASTA
donnees_fold = traiter_fasta_fold(fichier_fold)

# Step 4: Exécuter RNAfold sur un fichier FASTA pour prédire la structure secondaire
fichier_fasta = ...  # Chemin vers le fichier FASTA contenant les séquences
fichier_output = ...  # Chemin vers le fichier de sortie pour les prédictions de structure secondaire
executer_rnafold(fichier_fasta, fichier_output)

# Utiliser les données pour construire le jeu de données X et les étiquettes y
data1 = ...  # Données pour la classe 1
data2 = ...  # Données pour la classe 2
X, y = construire_dataset(data1, data2)

# Évaluer et entraîner un classifieur avec validation croisée
scores, classifieur = evaluer_cv_entrainer(X, y, cv=5, modele="lsvc", mesure='f1_weighted')
